# Vaccines Trade Network
### Author: Georgios Spyrou
### Date: 23/05/2020

<img src="https://www.our-voices.org.uk/assets/images/Network-diagram.png" width=720 height=400 />

### Project Puprose
Scope of this project is to construct the global trade network of vaccines around the globe for multiple years(current data span from 2017 to 2019. Through plotting and statistical analysis, we want to identify informative patterns on how different countries change their import/export activity of vaccines. 

### Part 1: Data Retrieval
In order to find and retrieve data we have utilised the UN Comtrade API (https://comtrade.un.org/Data/).This is a great place if someone wants to find data regarding exports/impors of countries around the world, pretty much for any product/service. The data are very well documented and visualized on the website, and it's easy to do some exploration  and locate data that seem interesting for your project.

Therefore, after I managed to identify the relevant data that I wanted to work with (vaccines for human medicine), I had to find a way to leverage the API to get my data. Even though the website allows us to download sample CSV files, if we wanted to do that for multiple years and countries it would take a lot of time. Hence, I have decided to approach the data retrival from a Python standpoint, in order to automate this task.

Before we jump to the part of how we are going to automate the data retrieval, it might worths it to explain what exactly we are aiming to retrieve.


The UN Comtrade database is giving us the opportunity to pick from a plethora of goods and services. My first task was to find the specific code that corresponds to human vaccines. After a bit of research we have found that this code is *300220*, which is a unique value that will allow us to only pick data for this good/product. After we located our product code, we have to decide on the time range that we want to pick data for. This can quickly get tricky as the database does not allow you to many years/countries all at once. But for now lets say that the scope of interest was *monthly* data from 2017 to 2019, for as many countries as possible - mainly because there are countries which do not seem to import/export vaccines or we do not have relevant information about them.

Now that are know pretty much what we are looking for, lets dive into creating some API calls in Python to automatically get the data.

In [3]:
import requests
import csv
import json
import time
import os

# Setting up the parameters for the API calls to receive the data

max_rec = 100000
output_fmt = 'csv'
trade_type = 'C'            # Commodities
frequency = 'M'             # Monthly
px = 'HS'                   # Classification for products
cc = 300220                 # Subcategory --> 300220 code for Vaccines
reporter = 'all'
partner = 'all'                 
rg ='all'

# Connection string to comtrade.un.org based on the parameters above
api_call_string = f'http://comtrade.un.org/api/get?max={max_rec}&type={trade_type}&freq={frequency}&px={px}&ps=year&r=reporter&p={partner}&rg={rg}&cc={cc}&fmt={output_fmt}'

After we have set up the string that will be used to do the appropriate API calls, it's time create a function that will use this string to retrieve the data and generate the csv files.

Note that this call will run for *each* country, therefore we will end up with a lot of csv files.

In [ ]:
def getDataCall(api_string: str, reporterid: str, reportername: str, year: int, out_folder: str) -> None:
    '''
    Create a .csv file that contains the data as received from  https://comtrade.un.org/Data/, for a specific year.

    Args:
    ----
        api_string: String that contains the URL for the API call. The string already contains all the paremeters required for the call.
        reporter: Specify Reporter country.
        year: Specify year of interest.
    Returns:
    -------
        None: The output is a .csv file that contains the data for a specified year.
    '''
    csv_by_year_out_loc = os.path.join(out_folder, f'{year}')
    if not os.path.exists(csv_by_year_out_loc):
        os.makedirs(name=csv_by_year_out_loc)

    api_string = api_string.replace('year', f'{year}').replace('reporter', f'{reporterid}')
    print(api_string)

    response = requests.get(url=api_string)

    if response.status_code != 200:
        print('Could not access the API!')
    else:
        decoded_data = response.content.decode('utf-8')
        csv_file = csv.reader(decoded_data.splitlines())
        datalines = list(csv_file)

        with open(os.path.join(csv_by_year_out_loc, f'Comtrade_Vaccines_Data_{reportername}_{year}.csv'), 'w', newline='') as f:
            writer = csv.writer(f, delimiter=',')
            writer.writerows(datalines)